In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data = pd.read_json('C:/Users/lizil/Dropbox/Job/Data/song.json')

In [3]:
data.head(10)

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey
5,XYDGPXHKLI,Sgt. Pepper Inner Groove,2015-06-28 08:25:26,147,2015-05-18,Texas
6,AYGDQROZIX,Hello Goodbye,2015-06-21 21:18:53,155,2015-05-19,Texas
7,RRRQYKTPNV,Cry For A Shadow,2015-06-05 08:29:21,171,2015-05-19,Illinois
8,HZICITYVPP,Revolution,2015-06-14 04:55:17,174,2015-05-19,Rhode Island
9,UYDPDSBKOY,Let It Be,2015-06-15 15:49:59,170,2015-05-19,Oregon


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
id                   4000 non-null object
song_played          4000 non-null object
time_played          4000 non-null object
user_id              4000 non-null int64
user_sign_up_date    4000 non-null object
user_state           4000 non-null object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [5]:
data['time_played']=pd.to_datetime(data['time_played'])
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])


# Question 1

What are the top 3 and the bottom 3 states in terms of number of users?

In [44]:
user_count = data.groupby('user_state').nunique().sort_values('user_id', ascending=False)

In [45]:
user_count.head(3)[['user_id']]

,user_id
user_state,
New York,23
California,21
Texas,15


In [46]:
user_count[user_count['user_id']==1][['user_id']]

,user_id
user_state,
New Mexico,1
Idaho,1
North Dakota,1
Connecticut,1
Iowa,1
Rhode Island,1
Nebraska,1
Arizona,1
Kansas,1


# Question 2

What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little

We define 'engagement' here as average of play times per state, per user

In [48]:
play_count = data[['user_id','user_state','id']].groupby(['user_id','user_state']).count().reset_index()

In [52]:
avg_play = play_count[['user_state','id']].groupby('user_state').mean().sort_values('id', ascending = False).reset_index()

In [55]:
avg_play.columns = ['state','avearge_user_play']

In [56]:
avg_play.head(3)

,state,avearge_user_play
0,Nebraska,36.000000
1,Alaska,29.000000
2,Mississippi,28.333333


In [58]:
avg_play.tail(3)

,state,avearge_user_play
38,Minnesota,10.5
39,Virginia,8.5
40,Kansas,8.0


# Question 3

The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?

In [89]:
def find_first_user(df):
    """ function to find the first user """
    idx = df['user_sign_up_date'].argmin()
    return df.loc[idx, ['user_id', 'user_sign_up_date']]

In [91]:
data.groupby('user_state').apply(find_first_user).sort_values(by='user_sign_up_date').reset_index()


,user_state,user_id,user_sign_up_date
0,Alabama,5,2015-05-01
1,Texas,7,2015-05-01
2,Oregon,1,2015-05-01
3,Ohio,3,2015-05-01
4,North Carolina,2,2015-05-01
5,New Mexico,4,2015-05-01
6,New Jersey,6,2015-05-01
7,Pennsylvania,11,2015-05-02
8,New York,19,2015-05-02
9,Minnesota,8,2015-05-02


Above method is intuitive and straightforward. But has one potential issue. There could be multiple users sign up at the same day. In this case, we need to define a second layer judgement rule to find the first user. Which should be the first guy who listened the song.

In [92]:
min_list = data[['user_state','user_sign_up_date']].groupby('user_state').min().reset_index()

In [94]:
data1 = pd.merge(left=data, right=min_list, how='left', on='user_state')

In [98]:
data1=data1[data1['user_sign_up_date_x']==data1['user_sign_up_date_y']]

In [100]:
def find_first_listener(df):
    """ function to find the first user """
    idx = df['time_played'].argmin()
    return df.loc[idx, ['user_id', 'user_sign_up_date_x','time_played']]

In [101]:
data1.groupby('user_state').apply(find_first_listener).sort_values(by='user_sign_up_date_x').reset_index()


,user_state,user_id,user_sign_up_date_x,time_played
0,Alabama,5,2015-05-01,2015-06-03 12:28:55
1,Texas,7,2015-05-01,2015-06-01 19:22:48
2,Oregon,1,2015-05-01,2015-06-05 14:30:22
3,Ohio,3,2015-05-01,2015-06-01 19:18:04
4,North Carolina,2,2015-05-01,2015-06-01 20:57:54
5,New Mexico,4,2015-05-01,2015-06-01 05:22:30
6,New Jersey,6,2015-05-01,2015-06-01 13:30:02
7,Pennsylvania,11,2015-05-02,2015-06-05 15:32:32
8,New York,12,2015-05-02,2015-06-01 18:26:15
9,Minnesota,8,2015-05-02,2015-06-13 08:33:28


In [106]:
data1[['user_id','user_state']].groupby('user_state').nunique()

,user_id,user_state
user_state,,
Alabama,1,1
Alaska,1,1
Arizona,1,1
Arkansas,1,1
California,2,1
Colorado,2,1
Connecticut,1,1
Florida,2,1
Georgia,2,1


# Question 4

Build a function that takes as an input any of the songs in the data and returns the most
likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

Idea: 
1. find for each user, which song he/she listened. Put all records in big matrix
2. normalize each row. (make sure the sum of square of all values in the row equal to 1 finally)
3. Calculate the song matrix based on the record. Behind idea: the clustered songs played by most users will have higher value
4. Given a song, find the related song by ranking the value from high to low

In [107]:
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)

song_user.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
from sklearn.preprocessing import normalize

song_user_norm = normalize(song_user, axis=1) 

In [116]:
similarity = np.dot(song_user_norm, song_user_norm.T) 

In [117]:
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

In [133]:
similarity_df

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.263117,0.138675,0.148250,0.131559,0.299572,0.294174,0.098058,0.228802,0.200160,...,0.525213,0.113228,0.585429,0.278503,0.539411,0.087706,0.328897,0.377141,0.163430,0.000000
A Hard Day's Night,0.263117,1.000000,0.000000,0.000000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.157027,0.000000,0.050000,0.215003,0.074536,0.000000
A Saturday Club Xmas/Crimble Medley,0.138675,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANYTIME AT ALL,0.148250,0.000000,0.000000,1.000000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.176950,0.000000,0.000000,0.103835,0.000000,0.000000
Across The Universe,0.131559,0.100000,0.000000,0.000000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.104685,0.000000,0.000000,0.061430,0.000000,0.000000
All My Loving,0.299572,0.146385,0.000000,0.164957,0.097590,1.000000,0.218218,0.218218,0.145479,0.089087,...,0.198946,0.000000,0.369129,0.056344,0.280945,0.195180,0.243975,0.209822,0.072739,0.000000
All You Need Is Love,0.294174,0.111803,0.000000,0.094491,0.000000,0.218218,1.000000,0.000000,0.166667,0.000000,...,0.199431,0.000000,0.273635,0.193649,0.234082,0.111803,0.111803,0.274721,0.166667,0.000000
And Your Bird Can Sing,0.098058,0.000000,0.000000,0.125988,0.000000,0.218218,0.000000,1.000000,0.222222,0.000000,...,0.189934,0.000000,0.265343,0.172133,0.195069,0.149071,0.223607,0.137361,0.000000,0.000000
BAD BOY,0.228802,0.000000,0.000000,0.000000,0.000000,0.145479,0.166667,0.222222,1.000000,0.000000,...,0.113961,0.192450,0.199007,0.172133,0.195069,0.000000,0.000000,0.091574,0.111111,0.000000


In [141]:
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df

In [144]:
df = find_topk('Eight Days A Week', similarity_df, 15)

In [145]:
df

,Song,Similarity
0,Come Together,0.417399
1,Hey Jude,0.398410
2,While My Guitar Gently Weeps,0.381503
3,Penny Lane,0.380693
4,A Day In The Life,0.375960
5,Let It Be,0.349749
6,Get Back,0.341918
7,Revolution,0.338261
8,I've Got A Feeling,0.335083
9,We Can Work It Out,0.332900
